# Optimal Draft Kings Lineup Selection


In this case, we will look at finding the optimal Draft Kings lineup. Let's take a look at the data we have.

In [59]:
import pandas as pd
import numpy as np
from gurobipy import *

In creating a Draft Kings lineup, you have a budget of 50,000 and you must select the following team:

- 1 QB
- 2 RBs
- 3 WRs
- 1 TE
- 1 Flex (RB, WR, TE)
- 1 D

Write a function that reads in the data and deletes any rows that have a player with a salary that <=0.

HINT:  Use the delimiter = ";" parameter within read_csv to read in the .txt file

In [60]:
def Read_Data():
    
    df = pd.read_csv("data/YTD_2018.txt", delimiter = ";")
    df_draft = df[df['Salary'] > 0]
    return df_draft


In [61]:
Read_Data()

,GID,Pos,Name,Team,Opponent,Home/Away,Salary,Salary Change,Points,GP,Pts/Game,Pts/G/$,Pts/G(alt),Bye week,YTD Salary H/L
0,1242,QB,"Fitzpatrick, Ryan",tam,pit,H,6000,500,79.26,2,39.63,6.61,33.98,5,H
1,5459,WR,"Thomas, Michael",no,atl,A,8900,300,73.90,2,36.95,4.15,31.90,6,H
2,1523,QB,"Mahomes II, Patrick",kan,sfo,H,7000,900,70.18,2,35.09,5.01,41.84,12,H
3,5485,WR,"Hill, Tyreek",kan,sfo,H,8500,900,65.30,2,32.65,3.84,20.00,12,H
4,5562,RB,"Kamara, Alvin",no,atl,A,9500,0,64.00,2,32.00,3.37,17.90,6,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,5451,RB,"Williams, Jonathan",no,atl,A,3600,-200,-0.10,2,-0.05,-0.01,-0.10,6,L
401,1241,QB,"Cassel, Matt",det,ne,H,4400,-200,-0.54,1,-0.54,-0.12,0.00,6,L
402,1522,QB,"Peterman, Nathan",buf,min,A,4000,-300,-0.94,1,-0.94,-0.24,0.00,11,L
403,5430,WR,"Clay, Kaelin",nyg,hou,A,3000,0,-1.00,2,-0.50,-0.17,0.00,9,NaN


In [62]:
assert Read_Data().shape == (399,15)

Write an integer program to choose that maximizes the "Pts/Game" total.  Return the optimal objective value.

In [63]:
def Get_Opt_Lineup():
    
    df_draft = Read_Data()
    l = Model()
    s = {}
    qb = {}
    rb = {}
    wr = {}
    te = {}
    flex = {}
    d = {}
    sal = {}
    flex_pos = ["RB","WR","TE"]
    for index in df_draft.index:
        s[index] = df_draft.loc[index,"Pts/Game"]
        sal[index] = df_draft.loc[index,"Salary"]
        if df_draft.loc[index,"Pos"] == "QB":
            qb[index] = l.addVar(lb=0,ub=1,vtype=GRB.BINARY,name="qb_%i" %index)
        if df_draft.loc[index,"Pos"] == "RB":
            rb[index] = l.addVar(lb=0,ub=1,vtype=GRB.BINARY,name="rb_%i" %index)
        if df_draft.loc[index,"Pos"] == "WR":
            wr[index] = l.addVar(lb=0,ub=1,vtype=GRB.BINARY,name="wr_%i" %index)
        if df_draft.loc[index,"Pos"] == "TE":
            te[index] = l.addVar(lb=0,ub=1,vtype=GRB.BINARY,name="te_%i" %index)
        if df_draft.loc[index,"Pos"] in flex_pos:
            flex[index] = l.addVar(lb=0,ub=1,vtype=GRB.BINARY,name="f_%i" %index)
        if df_draft.loc[index,"Pos"] == "D":
            d[index] = l.addVar(lb=0,ub=1,vtype=GRB.BINARY,name="d_%i" %index)
        
    l.setObjective(quicksum(s[i]*qb[i] for i in qb) + quicksum(s[i]*rb[i] for i in rb)\
                   +quicksum(s[i]*wr[i] for i in wr) + quicksum(s[i]*te[i] for i in te)\
                    +quicksum(s[i]*d[i] for i in d) + quicksum(s[i]*flex[i] for i in flex), GRB.MAXIMIZE)
    
    l.addConstr(quicksum(qb[i] for i in qb) == 1)
    l.addConstr(quicksum(rb[i] for i in rb) == 2)
    l.addConstr(quicksum(wr[i] for i in wr) == 3)
    l.addConstr(quicksum(te[i] for i in te) == 1)
    l.addConstr(quicksum(flex[i] for i in flex) == 1)
    l.addConstr(quicksum(d[i] for i in d) == 1)
    for i in flex:
        if i in rb:
            l.addConstr(flex[i]+rb[i] <= 1)
        if i in wr:
            l.addConstr(flex[i]+wr[i] <= 1)
        if i in te:
            l.addConstr(flex[i]+te[i] <= 1)
    l.addConstr(quicksum(sal[i]*qb[i] for i in qb) \
                + quicksum(sal[i]*rb[i] for i in rb) \
                + quicksum(sal[i]*wr[i] for i in wr) \
                + quicksum(sal[i]*flex[i] for i in flex) \
                + quicksum(sal[i]*te[i] for i in te) \
                + quicksum(sal[i]*d[i] for i in d) <= 50000)

    l.setParam('OutputFlag', False)
    l.optimize()
    return l.objVal
    
        

In [64]:
assert np.isclose(Get_Opt_Lineup(),244.63)